In [ ]:
!pip install --upgrade pip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/

In [ ]:
!pip install tensorflow scikit-learn imutils matplotlib numpy argparse

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [ ]:
%cd /content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/
print(os.getcwd())

dataset_path=os.getcwd()+"/data"
print(dataset_path)
model_path=os.getcwd()+"/model/drowsiness_model"
print(model_path)
plot_path=os.getcwd()+"/plot"
print(plot_path)

In [ ]:

INIT_LR = 1e-4
EPOCHS = 25
BS = 32

In [ ]:

print("Loading images...")
imagePaths = list(paths.list_images(dataset_path))
print(imagePaths)
data = []
labels = []

In [ ]:
import os
import numpy as np

data_cache_path = '/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/cache/data.npy'
labels_cache_path = '/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/cache/labels.npy'

if not (os.path.exists(data_cache_path) and os.path.exists(labels_cache_path)):

    for imagePath in imagePaths:
        label = imagePath.split("/")[-2]

        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(label)

    data_np = np.array(data)
    labels_np = np.array(labels)

    np.save(data_cache_path, data_np)
    np.save(labels_cache_path, labels_np)

    print("Data and labels have been saved to cache files.")
else:
    print("Cache files already exist. Skipping data processing.")


In [ ]:
data = np.load('/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/cache/data.npy')
labels = np.load('/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/cache/labels.npy')

data = data.astype("float32")
print(data);
print(labels);
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.01, stratify=labels, random_state=42)


In [ ]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
baseModel = MobileNetV2(weights="imagenet",
                        include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
print((len(trainX)))
print(BS)
print(len(trainX) // BS)

In [ ]:
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

print("[INFO] saving drowsiness detector model... path: %s"%(model_path+".h5"))
model.save(model_path+".h5")
model.save(model_path+'my_model.keras')

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont

image_path = "/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/testing/image.png"
image = Image.open(image_path)

image = image.resize((model.input_shape[1], model.input_shape[2]))

image_array = np.array(image)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)
print(predictions)

class_names = ['Awake', 'Drowsy']
max_index = np.argmax(predictions[0])
max_class_name = class_names[max_index]
max_probability = predictions[0][max_index]

draw = ImageDraw.Draw(image)
font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", 16)
max_probability_percentage = max_probability * 100
max_probability_percentage = round(max_probability_percentage, 2)
text = f"{max_class_name} ({max_probability_percentage}%)"
text_size = draw.textsize(text, font)
text_position = (10, 10) 

background_position = (text_position[0] - 5, text_position[1] - 5,
                       text_position[0] + text_size[0] + 5, text_position[1] + text_size[1] + 5)
draw.rectangle(background_position, fill = "red" if max_index == 1 else "green")
draw.text(text_position, text, fill="white", font=font)

output_path = "/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/testoutputs/image_with_text.png"
image.save(output_path)
print(f"Image saved to {output_path}")


In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os
import tensorflow as tf  
import time

model = tf.keras.models.load_model("/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/model/drowsiness_model.h5")
input_dir = "/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/testing"
output_dir = "/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/example234"

os.makedirs(output_dir, exist_ok=True)

class_names = ['Awake', 'Drowsy'] 

font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
font_size = 16
font = ImageFont.truetype(font_path, font_size)

start_time = time.time()

for filename in os.listdir(input_dir):
    if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
        try:
            image_path = os.path.join(input_dir, filename)
            image = Image.open(image_path)

            if image.mode != 'RGB':
                image = image.convert('RGB')

            image = image.resize((model.input_shape[1], model.input_shape[2]))

            image_array = np.array(image)
            image_array = np.expand_dims(image_array, axis=0)

            predictions = model.predict(image_array)
            print(predictions)
            max_index = np.argmax(predictions[0])
            max_class_name = class_names[max_index]
            max_probability = predictions[0][max_index]
            max_probability_percentage = max_probability * 100
            max_probability_percentage = round(max_probability_percentage, 2)

            draw = ImageDraw.Draw(image)
            text = f"{max_class_name} ({max_probability_percentage}%)"
            text_size = draw.textsize(text, font)
            text_position = (10, 10) 

            background_position = (text_position[0] - 5, text_position[1] - 5,
                                   text_position[0] + text_size[0] + 5, text_position[1] + text_size[1] + 5)
            draw.rectangle(background_position, fill="red" if max_index == 1 else "green")
            draw.text(text_position, text, fill="white", font=font)

            output_path = os.path.join(output_dir, filename)
            image.save(output_path)
            print(f"Image saved to {output_path}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")
end_time = time.time()

inference_time = end_time - start_time
print(f'Inference Time: {inference_time} seconds')

with open('inference_time.txt', 'w') as file:
    file.write(f'Inference Time: {inference_time} seconds\n')

In [ ]:
model_json = model.to_json()
json_file_path = "/content/gdrive/MyDrive/Drowsiness_detection_using_tensorflow/model/model.json"
with open(json_file_path, "w") as json_file:
    json_file.write(model_json)

In [ ]:
N = EPOCHS
plt.style.use("ggplot")

plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(os.path.join(plot_path, "loss.png"))

plt.figure()
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Training Val_Loss")
plt.xlabel("Epoch")
plt.ylabel("Val_Loss")
plt.legend(loc="lower left")
plt.savefig(os.path.join(plot_path, "val_loss.png"))

plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig(os.path.join(plot_path, "accuracy.png"))

plt.figure()
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Val_Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Val_Accuracy")
plt.legend(loc="lower left")
plt.savefig(os.path.join(plot_path, "val_accuracy.png"))

plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Loss/Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(os.path.join(plot_path, "all_plots.png"))